In [5]:
!pip install ipykernel



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install google-api-python-client google-auth

  Using cached google_api_python_client-2.187.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached google_auth-2.45.0-py2.py3-none-any.whl.metadata (6.8 kB)
  Using cached httplib2-0.31.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached google_auth_httplib2-0.3.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached google_api_core-2.28.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached uritemplate-4.2.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached cachetools-6.2.4-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached googleapis_common_protos-1.72.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached protobuf-6.33.2-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached proto_plus-1.27.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  U


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import json
!pip install pandas
import pandas as pd
import time
import re
from configparser import ConfigParser
!pip install tqdm
from tqdm.notebook import tqdm

# Import your local modules
# Make sure this notebook is in the root folder!
from modules.perspective_analyzer import PerspectiveAnalyzer

# 1. Load Config
config = ConfigParser()
config.read('config.ini')
PERSPECTIVE_JSON = config.get('API_KEYS', 'PERSPECTIVE_JSON_PATH')

# 2. Initialize Perspective API
perspective = PerspectiveAnalyzer(service_account_json=PERSPECTIVE_JSON)

# 3. Helper to clean pasted JSON (removes markdown ```json ... ```)
def clean_json_input(text_input):
    try:
        # If user pasted a python string representation or direct JSON
        return json.loads(text_input)
    except:
        # Clean markdown code blocks
        if "```" in text_input:
            text_input = re.split(r"```(?:json)?", text_input)[1].split("```")[0]
        return json.loads(text_input)

print("✅ Setup Complete. Perspective API Ready.")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Perspective API initialized successfully (Hindi Mode).
✅ Setup Complete. Perspective API Ready.


In [4]:
import os
import json
from modules.video_processor import VideoProcessor

VIDEO_FILENAME = "chunk4.mp4"
INPUT_VIDEO_PATH = os.path.join('data', 'input_videos', VIDEO_FILENAME)
OUTPUT_JSON_PATH = f"output/{os.path.splitext(VIDEO_FILENAME)[0]}_raw_transcript.json"

if os.path.exists(OUTPUT_JSON_PATH):
    print(f"✅ Transcript already exists at: {OUTPUT_JSON_PATH}")
    print("You can skip to the next cell!")
else:
    print(f"⏳ File not found. Starting Transcription & Diarization for: {INPUT_VIDEO_PATH}")
    print("This will take some time (loading models + processing)...")
    
    vp = VideoProcessor()
    
    raw_utterances = vp.process_video(INPUT_VIDEO_PATH)
    
    os.makedirs('output', exist_ok=True)
    with open(OUTPUT_JSON_PATH, 'w', encoding='utf-8') as f:
        json.dump(raw_utterances, f, ensure_ascii=False, indent=2)
        
    print(f"🎉 Success! Transcript saved to: {OUTPUT_JSON_PATH}")
    print("Now run the next cell to load it.")

ModuleNotFoundError: No module named 'torch'

In [12]:
VIDEO_NAME = "chunk5"
INPUT_FILE = f"output/{VIDEO_NAME}_raw_transcript.json"

if not os.path.exists(INPUT_FILE):
    print(f"❌ Error: File not found: {INPUT_FILE}")
    print("Please run main.py once to generate the raw transcript first.")
else:
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        raw_utterances = json.load(f)
    print(f"✅ Loaded {len(raw_utterances)} utterances from {INPUT_FILE}")

✅ Loaded 25 utterances from output/chunk5_raw_transcript.json


In [13]:
simplified_input = [
    {"id": i, "speaker": u["speaker"], "text": u["text"]} 
    for i, u in enumerate(raw_utterances)
]

prompt_pass_1 = f"""
You are an expert South Asian political analyst.

INPUT: A video transcript in HINDI SCRIPT (Devanagari) or URDU.
CONTEXT: The speakers are from PAKISTAN.

TASKS:
1. **CORRECT**: Fix spelling/grammar errors.
2. **TRANSLATE**: Translate the text to proper HINDI (Devanagari script) so it can be analyzed for toxicity.
3. **IDENTIFY ENTITIES**:
   - "source_entity": Speaker Name/Role.
   - "source_party": Political affiliation (PTI, PMLN, PPP, Army, etc.).
   - "target_entity": Who they are criticizing.
   - "target_party": Target's affiliation.
4. **FLAG TOXICITY**: If the text contains insults, threats, or severe criticism, set "check_toxicity": true.

OUTPUT FORMAT (Strict JSON List):
[
  {{
    "id": 0,
    "hindi_text": "...", 
    "source_entity": "...",
    "source_party": "...",
    "target_entity": "...",
    "target_party": "...",
    "check_toxicity": true/false
  }},
  ...
]

RAW TRANSCRIPT:
{json.dumps(simplified_input, ensure_ascii=False)}
"""

print("👇 COPY THE TEXT BELOW THIS LINE 👇")
print("-" * 30)
print(prompt_pass_1)
print("-" * 30)

👇 COPY THE TEXT BELOW THIS LINE 👇
------------------------------

You are an expert South Asian political analyst.

INPUT: A video transcript in HINDI SCRIPT (Devanagari) or URDU.
CONTEXT: The speakers are from PAKISTAN.

TASKS:
1. **CORRECT**: Fix spelling/grammar errors.
2. **TRANSLATE**: Translate the text to proper HINDI (Devanagari script) so it can be analyzed for toxicity.
3. **IDENTIFY ENTITIES**:
   - "source_entity": Speaker Name/Role.
   - "source_party": Political affiliation (PTI, PMLN, PPP, Army, etc.).
   - "target_entity": Who they are criticizing.
   - "target_party": Target's affiliation.
4. **FLAG TOXICITY**: If the text contains insults, threats, or severe criticism, set "check_toxicity": true.

OUTPUT FORMAT (Strict JSON List):
[
  {
    "id": 0,
    "hindi_text": "...", 
    "source_entity": "...",
    "source_party": "...",
    "target_entity": "...",
    "target_party": "...",
    "check_toxicity": true/false
  },
  ...
]

RAW TRANSCRIPT:
[{"id": 0, "speaker": "

In [14]:
# PASTE YOUR JSON RESPONSE HERE
llm_response_text_1 = """

[
  {
    "id": 0,
    "hindi_text": "[संगीत]",
    "source_entity": "N/A",
    "source_party": "N/A",
    "target_entity": "N/A",
    "target_party": "N/A",
    "check_toxicity": false
  },
  {
    "id": 1,
    "hindi_text": "एक कड़े रुख में जो आज सामने आया है...",
    "source_entity": "वसीम बादामी (संभावित)",
    "source_party": "मीडिया",
    "target_entity": "मौलाना फजलुर रहमान",
    "target_party": "JUI-F",
    "check_toxicity": false
  },
  {
    "id": 3,
    "hindi_text": "वह बहुत सख्त लहजे में आया है, वह है जनाब मौलाना फजलुर रहमान साहब का, जिनका सारांश यह था कि उनकी जमात के रहनुमाओं से बात करेंगे कि उन्होंने इसे पूरी तरह खारिज कर दिया है। संस्थानों से नाराजगी जाहिर की है, उन्होंने चुनाव को धांधली वाला कहा और कहा कि 2018 से भी ज्यादा धांधली इस चुनाव में हुई है। मौलाना साहब ने यहाँ तक कहा कि हम तो संसदीय राजनीति छोड़ने के बारे में भी सोच रहे हैं। हमारे साथ अली मोहम्मद खान साहब मौजूद हैं जो किसी परिचय के मोहताज नहीं हैं। अली मोहम्मद खान साहब यह चुनाव बहुत अच्छे मार्जिन से जीते हैं, मर्दान से जीते हैं। अगर मैं राउंड ऑफ करूँ तो उन्होंने करीब एक लाख दो हजार वोट लिए हैं। उनके जो प्रतिद्वंद्वी उम्मीदवार दूसरे नंबर पर आए, वो एएनपी (ANP) के थे जिन्होंने चौबीस हजार वोट लिए। यानी 24,000 बनाम 1,02,000...",
    "source_entity": "वसीम बादामी (संभावित)",
    "source_party": "मीडिया",
    "target_entity": "मौलाना फजलुर रहमान",
    "target_party": "JUI-F",
    "check_toxicity": false
  },
  {
    "id": 4,
    "hindi_text": "मेरे प्रतिद्वंद्वी मोहतरम जनाब अहमद खान बहादुर साहब ने तैंतीस हजार (33,000) वोट लिए हैं।",
    "source_entity": "अली मोहम्मद खान",
    "source_party": "PTI",
    "target_entity": "अहमद खान बहादुर",
    "target_party": "ANP",
    "check_toxicity": false
  },
  {
    "id": 5,
    "hindi_text": "मेरे पास 30,917 लिखा है, लेकिन फिर भी आपकी लीड बहुत ज्यादा है। यह अच्छी बात है कि पिछली बार भी आपका कड़ा मुकाबला मौलाना साहब की पार्टी के साथ था, लेकिन आप लोग ब्रेक में साथ मस्जिद में नमाज पढ़ने जाते थे और कहते थे कि हमारा सारा मतभेद राजनीति का है, कोई व्यक्तिगत दुश्मनी नहीं है। यह बहुत अच्छी परंपरा है, मैं उम्मीद करता हूँ कि यह वतन-ए-अजीज में फले-फूले कि हम वैचारिक मतभेद डटकर करें लेकिन व्यक्तिगत लड़ाई न हो। अभी भी आपने जिस तरह सम्मान से अपने प्रतिद्वंद्वी का नाम लिया, मुझे अच्छा लगा।",
    "source_entity": "वसीम बादामी (संभावित)",
    "source_party": "मीडिया",
    "target_entity": "N/A",
    "target_party": "N/A",
    "check_toxicity": false
  },
  {
    "id": 8,
    "hindi_text": "हमारे पश्तून नेता रहे हैं अब्दुल गफ्फार खान बाबा, जिन्हें बाचा खान कहते हैं। एएनपी (ANP) उन्हें अपना लीडर मानती है, लेकिन मैं समझता हूँ हर पश्तून उन्हें बड़ा लीडर मानता है। भले ही कुछ वैचारिक मतभेद रहे हों, लेकिन पाकिस्तान बनने के बाद उन्होंने इसे स्वीकार किया। उनका एक मशहूर कथन है कि 'खिदमत (सेवा) में कोई मतभेद नहीं होता, मतभेद तो स्वार्थ से पैदा होता है'।",
    "source_entity": "अली मोहम्मद खान",
    "source_party": "PTI",
    "target_entity": "बाचा खान/ANP",
    "target_party": "ANP",
    "check_toxicity": false
  },
  {
    "id": 12,
    "hindi_text": "एक वकील के तौर पर मेरी ट्रेनिंग भी इसमें मदद करती है क्योंकि हम अदालतों में खूब लड़ते हैं लेकिन बार रूम में साथ बैठकर चाय पीते हैं। तो यह समझ आ जाता है कि यह दुश्मनी नहीं बल्कि राजनीतिक मतभेद है। मर्दान में जो अभी चुनाव हुआ, हम तीनों नेशनल असेंबली की सीटें जीते हैं—आतिफ खान साहब, मुजाहिद खान साहब और मैं। हमारी प्रांतीय असेंबली की भी आठ की आठ सीटें जीते हैं, यानी 100% रिजल्ट है। मेरी जानकारी में मर्दान में ऐसा कोई वाकया नहीं हुआ जहाँ दो राजनीतिक विरोधी आपस में लड़ मरे हों।",
    "source_entity": "अली मोहम्मद खान",
    "source_party": "PTI",
    "target_entity": "N/A",
    "target_party": "N/A",
    "check_toxicity": false
  },
  {
    "id": 17,
    "hindi_text": "अली मोहम्मद खान साहब, अब यह बताइए कि क्या आपकी जमात का यह मानना है कि आप केंद्र (وفاق) और पंजाब में विपक्ष में बैठेंगे? या आप कहते हैं कि नहीं जनाब, हम केंद्र और पंजाब में भी सरकार बना सकते हैं?",
    "source_entity": "वसीम बादामी (संभावित)",
    "source_party": "मीडिया",
    "target_entity": "PTI",
    "target_party": "PTI",
    "check_toxicity": false
  },
  {
    "id": 18,
    "hindi_text": "इसका जवाब यह है कि अगर हमें हमारा हक मिल जाता है, तो हक क्या है? नंबर एक: 40 से 50 के करीब हमारी सीटें हैं जो फॉर्म 45 (Form 45) के आधार पर हमारी हैं। अगर बिना किसी हेरा-फेरी के सिर्फ फॉर्म 45 को जोड़ लिया जाए, तो आप असली नतीजे पर पहुँच जाएंगे। हम मुल्क की सबसे बड़ी जमात हैं, हमने 93 सीटें ली हैं। मरियम बीबी ने एक बात कही थी कि 'लोटों' (दलबदलुओं) की जगह बाथरूम में है, मैं कहता हूँ लोटों की जगह मस्जिद में है वज़ू करने के लिए। लेकिन फिर उन्होंने उसी 'लोटे' को साथ खड़ा करके तस्वीर खिंचवाई। मैं समझता हूँ यह उनके जीवन की सबसे दुखद तस्वीर है क्योंकि उन्होंने अपनी संसदीय राजनीति की शुरुआत एक दलबदलू के साथ तस्वीर खिंचवाकर की। उन्होंने खुद 'वोट को इज्जत दो' के फलसफे का जनाज़ा निकाला है।",
    "source_entity": "अली मोहम्मद खान",
    "source_party": "PTI",
    "target_entity": "मरियम नवाज",
    "target_party": "PMLN",
    "check_toxicity": true
  },
  {
    "id": 20,
    "hindi_text": "अगर हमें हमारी 93 सीटें मिलें, जो असल में 100 से ऊपर जाएंगी, तो हमारे पीछे एक बहुत बड़ा जनादेश खड़ा है। क्या आरक्षित सीटों (Reserved Seats) पर महिलाओं का हक नहीं बनता? इसके लिए हम एक-दो दिनों में एक राजनीतिक दल का चुनाव करेंगे ताकि हमें हमारी आरक्षित सीटें मिल सकें।",
    "source_entity": "अली मोहम्मद खान",
    "source_party": "PTI",
    "target_entity": "N/A",
    "target_party": "N/A",
    "check_toxicity": false
  },
  {
    "id": 24,
    "hindi_text": "अभी ऐसी कोई स्थिति (फॉरवर्ड ब्लॉक) बनती नज़र नहीं आ रही, लेकिन अतीत में ऐसी संभावनाएं रही हैं। किसी दूसरी पार्टी के टिकट पर जीतकर हमारी जमात में शामिल होना अनैतिक है। मरियम बीबी और उनके वालिद (नवाज शरीफ) ने 'वोट को इज्जत दो' की बात की थी, लेकिन स्थिति ऐसी है कि तहरीक-ए-इंसाफ अपने दम पर सरकार बना सकती है अगर हमारे साथ न्याय हो। नून लीग (PMLN) या पीपल्स पार्टी में फिलहाल ऐसी कोई हलचल नहीं दिख रही। खान साहब को इन चीजों (दलबदलुओं) की परवाह नहीं है।",
    "source_entity": "अली मोहम्मद खान",
    "source_party": "PTI",
    "target_entity": "मरियम नवाज / नवाज शरीफ",
    "target_party": "PMLN",
    "check_toxicity": false
  }
]

"""

# Parse and Validate
try:
    pass_1_data = clean_json_input(llm_response_text_1)
    print(f"✅ Successfully parsed {len(pass_1_data)} items.")
except Exception as e:
    print(f"❌ JSON Error: {e}")

✅ Successfully parsed 11 items.


In [15]:
# Merge LLM results with Original Timestamps
combined_data = []

print("Running Perspective API Analysis...")

for i, raw in enumerate(raw_utterances):
    # Find matching LLM result
    llm_res = next((item for item in pass_1_data if item.get("id") == i), {})
    
    # Construct combined object
    merged = {
        "id": i,
        "start": raw['start'],
        "end": raw['end'],
        "speaker": raw['speaker'],
        "original_text": raw['text'],
        "hindi_text": llm_res.get("hindi_text", raw['text']),
        "source_entity": llm_res.get("source_entity", "Unknown"),
        "source_party": llm_res.get("source_party", "None"),
        "target_entity": llm_res.get("target_entity", "None"),
        "target_party": llm_res.get("target_party", "None"),
        "perspective_scores": {}
    }
    
    # Check Toxicity if flagged
    if llm_res.get("check_toxicity", False):
        scores = perspective.get_scores(merged["hindi_text"])
        if scores:
            merged["perspective_scores"] = scores
        time.sleep(1.1) # Rate limit safety
        
    combined_data.append(merged)

print("✅ Scoring Complete.")

Running Perspective API Analysis...
✅ Scoring Complete.


In [16]:
# We need to construct the prompt with the NEW scores we just got
prompt_pass_2 = f"""
You are a political scientist analyzing polarization.

INPUT: Hindi Utterances with Toxicity Scores (Perspective API).

FRAMEWORK DEFINITIONS:
1. **Personality Polarization**: Attacks on a leader's character.
   - Codes: Strawman, Offensive Language, Absolutism, Invalidation, Vilification/Defamation, Threats, Moral Superiority.
2. **Party Polarization**: Attacks on a group/party.
   - Codes: Strawman, Offensive Language, Absolutism, Invalidation, Otherization, Threats, Moral Superiority.
3. **Issue Polarization**: Disagreement on policies/facts.
   - Codes: Strawman, Absolutism, Invalidation, Otherization, Fact Denial.

TASK:
For each utterance, determine which DOMAIN and CATEGORY it belongs to.
If multiple apply, create a list of annotations.

GUIDELINES:
- High 'INSULT' or 'THREAT' score (>0.6) = Insult/Threat.
- High 'TOXICITY' but low Insult = Criticism/Abuse.

OUTPUT FORMAT (JSON List):
[
  {{
     "id": 0,
     "annotations": [
       {{
         "type": "Personality Polarization", 
         "code": "Vilification",
         "justification": "..."
       }}
     ]
  }}
]

INPUT DATA:
{json.dumps(combined_data, ensure_ascii=False)}
"""

print("👇 COPY THE TEXT BELOW THIS LINE 👇")
print("-" * 30)
print(prompt_pass_2)
print("-" * 30)

👇 COPY THE TEXT BELOW THIS LINE 👇
------------------------------

You are a political scientist analyzing polarization.

INPUT: Hindi Utterances with Toxicity Scores (Perspective API).

FRAMEWORK DEFINITIONS:
1. **Personality Polarization**: Attacks on a leader's character.
   - Codes: Strawman, Offensive Language, Absolutism, Invalidation, Vilification/Defamation, Threats, Moral Superiority.
2. **Party Polarization**: Attacks on a group/party.
   - Codes: Strawman, Offensive Language, Absolutism, Invalidation, Otherization, Threats, Moral Superiority.
3. **Issue Polarization**: Disagreement on policies/facts.
   - Codes: Strawman, Absolutism, Invalidation, Otherization, Fact Denial.

TASK:
For each utterance, determine which DOMAIN and CATEGORY it belongs to.
If multiple apply, create a list of annotations.

GUIDELINES:
- High 'INSULT' or 'THREAT' score (>0.6) = Insult/Threat.
- High 'TOXICITY' but low Insult = Criticism/Abuse.

OUTPUT FORMAT (JSON List):
[
  {
     "id": 0,
     "ann

In [ ]:
# PASTE YOUR FINAL JSON RESPONSE HERE
llm_response_text_2 = """
[
  {
    "id": 3,
    "annotations": [
      {
        "type": "Issue Polarization",
        "code": "Invalidation",
        "justification": "The speaker conveys Maulana Fazlur Rehman's claim that the election was completely rigged, more so than in 2018, thereby invalidating the entire electoral process and the legitimacy of the results."
      },
      {
        "type": "Issue Polarization",
        "code": "Fact Denial",
        "justification": "The assertion that the election was 'nothing but rigging' functions as a denial of the official facts/results released by the institutions."
      }
    ]
  },
  {
    "id": 18,
    "annotations": [
      {
        "type": "Personality Polarization",
        "code": "Vilification",
        "justification": "The speaker uses the metaphor of 'carrying out the funeral' of a political philosophy to describe Maryam Nawaz's actions, which serves to defame her character and political integrity."
      },
      {
        "type": "Party Polarization",
        "code": "Invalidation",
        "justification": "The speaker challenges the legitimacy of the opposing party's seats by claiming they were obtained through 'hera-pheri' (manipulation) and contrasts them with 'Form 45' data."
      },
      {
        "type": "Personality Polarization",
        "code": "Invalidation",
        "justification": "The speaker labels the target as a 'Lota' (turncoat) and mockingly suggests their place is in a bathroom, aiming to invalidate their status as a legitimate parliamentary leader."
      }
    ]
  },
  {
    "id": 24,
    "annotations": [
      {
        "type": "Party Polarization",
        "code": "Moral Superiority",
        "justification": "The speaker explicitly labels the actions of the opposing party (PMLN) as 'unethical' while framing his own party's struggle as being based on 'justice' and 'merit'."
      },
      {
        "type": "Personality Polarization",
        "code": "Invalidation",
        "justification": "By referencing the 'Vote Ko Izzat Do' slogan and claiming the leaders have abandoned it, the speaker seeks to invalidate the core political narrative and sincerity of Maryam Nawaz and Nawaz Sharif."
      }
    ]
  }
]
"""

# Parse
try:
    pass_2_data = clean_json_input(llm_response_text_2)
    print(f"✅ Successfully parsed final annotations for {len(pass_2_data)} items.")
except Exception as e:
    print(f"❌ JSON Error: {e}")

✅ Successfully parsed final annotations for 10 items.


In [11]:
csv_rows = []

for item in combined_data:
    # Find classification
    llm_res = next((f for f in pass_2_data if f.get("id") == item["id"]), {})
    annotations = llm_res.get("annotations", [])
    
    base_row = {
        "start_time": item.get("start"),
        "end_time": item.get("end"),
        "speaker_label": item.get("speaker"),
        "original_text": item.get("original_text"),
        "hindi_translation": item.get("hindi_text"),
        "source_entity": item.get("source_entity"),
        "source_party": item.get("source_party"),
        "target_entity": item.get("target_entity"),
        "target_party": item.get("target_party"),
        "toxicity": item.get("perspective_scores", {}).get("TOXICITY", 0.0),
        "insult": item.get("perspective_scores", {}).get("INSULT", 0.0),
        "threat": item.get("perspective_scores", {}).get("THREAT", 0.0)
    }

    if not annotations:
        row = base_row.copy()
        row["polarization_type"] = "None"
        row["polarization_code"] = "Neutral"
        row["justification"] = "No polarization detected."
        csv_rows.append(row)
    else:
        for ann in annotations:
            row = base_row.copy()
            row["polarization_type"] = ann.get("type", "Unknown")
            row["polarization_code"] = ann.get("code", "Unknown")
            row["justification"] = ann.get("justification", "")
            csv_rows.append(row)

output_filename = f"output/{VIDEO_NAME}_manual_analysis.csv"
df = pd.DataFrame(csv_rows)

# Reorder columns
cols = [
    "start_time", "end_time", "speaker_label", 
    "polarization_type", "polarization_code", "justification",
    "original_text", "hindi_translation", 
    "source_entity", "source_party", "target_entity", "target_party",
    "toxicity", "insult", "threat"
]
existing_cols = [c for c in cols if c in df.columns]
df = df[existing_cols]

df.to_csv(output_filename, index=False, encoding='utf-8-sig')
print(f"🎉 Success! File saved to: {output_filename}")
df.head()

🎉 Success! File saved to: output/chunk4_manual_analysis.csv


,start_time,end_time,speaker_label,polarization_type,polarization_code,justification,original_text,hindi_translation,source_entity,source_party,target_entity,target_party,toxicity,insult,threat
0,0.24,21.76,SPEAKER_00,Personality Polarization,Invalidation,The host points out a 'huge contradiction' bet...,بہت شکریہ آتھر گفتگو آپ سن رہے ہیں دو ...,बहुत शुक्रिया अतहर। आप दो माननीय मेहमानों की ब...,मेजबान (Host),मीडिया,इमरान खान और अली अमीन गंडापुर,PTI,0.0,0.0,0.0
1,25.22,185.92,SPEAKER_01,Party Polarization,Invalidation,The speaker dismisses the legal cases regardin...,دیکھیں اب اس میں ایشو یہ ہے کہ یہاں پ...,"देखें, इसमें मुद्दा यह है कि यहाँ 9 मई की घटना...",अतहर काज़मी (Athar Kazmi),विश्लेषक (PTI समर्थक),"PMLN, PPP, मोहसिन नक़वी","PMLN, PPP, निर्दलीय/सरकार",0.0,0.0,0.0
2,25.22,185.92,SPEAKER_01,Personality Polarization,Vilification,The speaker refers to Mohsin Naqvi as a 'Vicer...,دیکھیں اب اس میں ایشو یہ ہے کہ یہاں پ...,"देखें, इसमें मुद्दा यह है कि यहाँ 9 मई की घटना...",अतहर काज़मी (Athar Kazmi),विश्लेषक (PTI समर्थक),"PMLN, PPP, मोहसिन नक़वी","PMLN, PPP, निर्दलीय/सरकार",0.0,0.0,0.0
3,25.22,185.92,SPEAKER_01,Issue Polarization,Fact Denial,The speaker challenges the state's factual acc...,دیکھیں اب اس میں ایشو یہ ہے کہ یہاں پ...,"देखें, इसमें मुद्दा यह है कि यहाँ 9 मई की घटना...",अतहर काज़मी (Athar Kazmi),विश्लेषक (PTI समर्थक),"PMLN, PPP, मोहसिन नक़वी","PMLN, PPP, निर्दलीय/सरकार",0.0,0.0,0.0
4,25.22,185.92,SPEAKER_01,Personality Polarization,Moral Superiority,The speaker claims Imran Khan is 'most popular...,دیکھیں اب اس میں ایشو یہ ہے کہ یہاں پ...,"देखें, इसमें मुद्दा यह है कि यहाँ 9 मई की घटना...",अतहर काज़मी (Athar Kazmi),विश्लेषक (PTI समर्थक),"PMLN, PPP, मोहसिन नक़वी","PMLN, PPP, निर्दलीय/सरकार",0.0,0.0,0.0
